In [1]:
from __future__ import print_function, division
import requests
import pandas as pd
import numpy as np
import os.path
from bs4 import BeautifulSoup
import patsy

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

%matplotlib inline

import statsmodels.api as sm
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
import statsmodels.formula.api as smf
from statsmodels.tools.data import _is_using_pandas, _is_recarray
from statsmodels.tools.validation import array_like
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv(r'C:\Users\andro\Metis\module2_repo\combined.csv')

In [3]:
#Limiting 'Pos' column to 2 characters

def char_len(x, fixed_n):
    if len(x) > fixed_n: 
        return x[:fixed_n]
    else:
        return x
    
df['Pos'] = df['Pos'].apply(lambda x: char_len(x,2))
df['Pos'] = df['Pos'].replace(['C-'], 'C')

In [4]:
#Creating dummies
dummy = pd.get_dummies(df['Pos'], drop_first=True)

In [5]:
df.Pos.unique()

array(['PF', 'PG', 'C', 'SG', 'SF', 'F', 'G'], dtype=object)

In [6]:
df = df.drop(columns=['Unnamed: 0','G','PF'])

In [7]:
#Merge dummies
df = df.merge(dummy, left_index=True,right_index=True)
df.head()

,Player,Pos,Age,Tm,GS,MP,FG,FGA,FG%,3P,...,TOV,PTS,YEAR,SALARY,F,G,PF,PG,SF,SG
0,Precious Achiuwa,PF,21,MIA,2,12.3,2.1,3.8,0.556,0.0,...,0.8,5.2,2021,2582160,0,0,1,0,0,0
1,Jaylen Adams,PG,24,MIL,0,2.6,0.1,1.1,0.125,0.0,...,0.0,0.3,2021,449115,0,0,0,1,0,0
2,Steven Adams,C,27,NOP,49,27.9,3.4,5.5,0.616,0.0,...,1.5,7.8,2021,29592695,0,0,0,0,0,0
3,Bam Adebayo,C,23,MIA,47,33.5,7.3,12.9,0.565,0.0,...,2.6,19.1,2021,5115492,0,0,0,0,0,0
4,LaMarcus Aldridge,C,35,TOT,23,25.9,5.4,11.4,0.473,1.2,...,1.0,13.5,2021,17628340,0,0,0,0,0,0


In [17]:
#FE: Adding Approximate Value
FGMi = df['FGA']-df['FG']
FTMi = df['FTA']-df['FT']
Credits = (df['PTS']+df['TRB']+df['AST']+df['STL']+df['BLK'])-(FGMi-FTMi)-df['TOV']
df['AV'] = (Credits**(.75))/21
df=df.fillna(0)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(df[['Age', 'MP', '3P', '2P', 'FT',
       'TRB', 'AST', 'STL', 'BLK', 'TOV', 
       'PTS', 'YEAR','F','G','PF','PG','SF','SG','AV']], df['SALARY'], test_size=0.2, random_state=10)

In [19]:
m = LinearRegression()
m.fit(X_train,y_train)
m.score(X_train,y_train)

0.5685378757996102

In [20]:
m.score(X_test,y_test)

0.5750267396467824

In [21]:
from sklearn.preprocessing import PolynomialFeatures
p = PolynomialFeatures(degree=2)
x_train_poly = p.fit_transform(X_train)
m.fit(x_train_poly,y_train)
m.score(x_train_poly,y_train)

0.6886302182469829

In [22]:
m.score(p.transform(X_test),y_test)

0.6817968457435686

In [23]:
lr_poly.coef_

NameError: name 'lr_poly' is not defined

In [ ]:
m = LassoCV()
p = PolynomialFeatures(degree=2)
X_train_poly = p.fit_transform(X_train)
s = StandardScaler()
X_train_poly_scaled = s.fit_transform(X_train_poly)
m.fit(X_train_poly_scaled,y_train)
m.score(X_train_poly_scaled,y_train)

In [ ]:
m.score(p.transform(X_test),y_test)

In [ ]:
selected_columns = ['Age', 'MP', '3P', '2P', 'FT',
       'TRB', 'AST', 'STL', 'BLK', 'TOV', 
       'PTS', 'YEAR']

np.random.seed(6)

X_train_collinear = X_train.loc[:,selected_columns]
lr_model_ridge = Ridge(alpha = 10000)
lr_model_ridge.fit(X_train_collinear, y_train)

list(zip(X_train_collinear.columns, lr_model_ridge.coef_))

In [ ]:
preds = m.predict(X_train)
ax = sns.jointplot(x=preds,y=y_train, kind='reg')
ax.fig.suptitle('NBA Player Salary Predicted By Performance Stats')
ax.fig.subplots_adjust(top=0.95)

In [ ]:
df['res'] = m.predict(df[['Age', 'MP', '3P', '2P', 'FT',
       'TRB', 'AST', 'STL', 'BLK', 'TOV', 
       'PTS', 'YEAR']])-df['SALARY']

df.sort_values(['res']).tail(10)

In [ ]:
m.coef_